In [1]:
import tensorflow as tf
from konlpy.tag import Twitter
import numpy as np
import gensim

In [2]:
def read_data(filename):    
    with open(filename, 'r',encoding='utf-8') as f:
        data = [line.split('\t') for line in f.read().splitlines()]        
        data = data[1:]   # header 제외 #    
    return data 
    
train_data = read_data('data/ratings_train3.txt')
test_data = read_data('data/ratings_test.txt')

In [3]:
twitter = Twitter() 

def tokenize(doc):

    return ['/'.join(t) for t in twitter.pos(doc, norm=True, stem=True)]

tokens = [tokenize(row[1]) for row in train_data]

In [4]:
model = gensim.models.Word2Vec(size=300, sg = 1, alpha=0.025, min_alpha=0.025, seed=410)
model.build_vocab(tokens)
model.train(tokens, model.corpus_count, epochs=model.epochs)

(934071, 1291235)

In [5]:
# '외계인/Noun' in model.wv.vocab

In [6]:
model.save('word2vec.model')

In [7]:
model = gensim.models.word2vec.Word2Vec.load('word2vec.model')

In [8]:
len(model.wv.vocab)

def convert2Vec(model, doc):  ## Convert corpus into vectors
    word_vec = []
    word_num = 0
    unk_num = 0
#     print('doc : ', doc)
    for sent in doc:
#         print('sent : ', sent)
        sub = []
        for word in sent:
            word_num += 1
            if(word in model.wv.vocab):
#                 print('word in sent')
                sub.append(model.wv[word])
            else:
                unk_num += 1
#                 print('word not in sent')
                sub.append(np.zeros(300)) ## used for OOV words
        word_vec.append(sub)
    print('word_num :', word_num, 'unk_num :', unk_num)
    return word_vec


In [9]:
tokens = [[tokenize(row[1]),int(row[2])] for row in train_data if tokenize(row[1]) != []]
tokens = np.array(tokens)
train_X = tokens[:,0]
train_Y = tokens[:,[-1]]
train_X = convert2Vec(model, train_X)
# train_Y = np.array([[y] for y in train_Y])
seq_length = [len(x) for x in train_X]
maxseq_length = max(seq_length)
maxseq_length


word_num : 258247 unk_num : 20510


76

In [10]:
def zero_padding(maxseq_length, seq):
#     print(len(seq))
    zero_pad = np.zeros((maxseq_length) * 300)
    seq_flat = np.reshape(seq, [-1])
#     print(len(seq_flat))
    zero_pad[0:len(seq) * 300] = seq_flat
#     print(len(zero_pad))
    return zero_pad
train_X = [zero_padding(maxseq_length, seq) for seq in train_X]
len(train_X[0])

22800

In [11]:
X = tf.placeholder(tf.float32, shape=[None, maxseq_length * 300])
Y = tf.placeholder(tf.float32, shape=[None, 1])

In [12]:
W = tf.Variable(tf.random_normal([maxseq_length * 300, 1]), name='weight1')
b = tf.Variable(tf.random_normal([1]), name='bias')

In [13]:
hypothesis = tf.sigmoid(tf.matmul(X, W) + b)

In [14]:
# cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) *
#                        tf.log(1 - hypothesis))
cost = -tf.reduce_mean(Y * tf.log(hypothesis + 1e-7) + (1 - Y) * tf.log(1 - hypothesis + 1e-7))

In [15]:
train = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)

In [16]:
predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))

In [20]:
training_epochs = 100
Batch_size = 1000
total_batch = int(len(train_X) / Batch_size)
save_path = './saved/model'
saver = tf.train.Saver()

with tf.Session() as sess:
    # Initialize TensorFlow variables
    sess.run(tf.global_variables_initializer())

    for epoch in range(training_epochs):

        avg_loss = 0
        for step in range(total_batch):

            train_batch_X = train_X[step*Batch_size : step*Batch_size+Batch_size]
            train_batch_Y = train_Y[step*Batch_size : step*Batch_size+Batch_size]
            
            cost_val, _ = sess.run([cost, train], feed_dict={X: train_batch_X, Y: train_batch_Y})
            avg_loss += cost_val
            acc = sess.run(accuracy , feed_dict={X: train_batch_X, Y: train_batch_Y})
            print('Batch : ', step + 1, '/', total_batch,
                  ', BCE in this minibatch: ', cost_val, 'accuracy: ', float(acc))

            # print("step:", '%04d' %(step+1), "loss = {:.6f} accuracy= {:.6f}".format(loss_, acc))
        acc = sess.run(accuracy , feed_dict={X: train_batch_X, Y: train_batch_Y})
        print('epoch:', epoch, ' train_loss:', float(avg_loss/total_batch), ' accuracy: ', float(acc))
        saver.save(sess, save_path, epoch)


Batch :  1 / 17 , BCE in this minibatch:  2.8841782 accuracy:  0.5070000290870667
Batch :  2 / 17 , BCE in this minibatch:  2.8314707 accuracy:  0.5130000114440918
Batch :  3 / 17 , BCE in this minibatch:  2.7278855 accuracy:  0.503000020980835
Batch :  4 / 17 , BCE in this minibatch:  2.8960571 accuracy:  0.5189999938011169
Batch :  5 / 17 , BCE in this minibatch:  2.7227921 accuracy:  0.5260000228881836
Batch :  6 / 17 , BCE in this minibatch:  2.5184107 accuracy:  0.5519999861717224
Batch :  7 / 17 , BCE in this minibatch:  2.7322488 accuracy:  0.5189999938011169
Batch :  8 / 17 , BCE in this minibatch:  2.9446533 accuracy:  0.5080000162124634
Batch :  9 / 17 , BCE in this minibatch:  2.6626778 accuracy:  0.5189999938011169
Batch :  10 / 17 , BCE in this minibatch:  2.676002 accuracy:  0.5210000276565552
Batch :  11 / 17 , BCE in this minibatch:  2.5970933 accuracy:  0.5299999713897705
Batch :  12 / 17 , BCE in this minibatch:  2.607522 accuracy:  0.5479999780654907
Batch :  13 / 17

Batch :  13 / 17 , BCE in this minibatch:  2.0498354 accuracy:  0.5730000138282776
Batch :  14 / 17 , BCE in this minibatch:  2.0891795 accuracy:  0.5690000057220459
Batch :  15 / 17 , BCE in this minibatch:  2.0109954 accuracy:  0.5889999866485596
Batch :  16 / 17 , BCE in this minibatch:  2.0786593 accuracy:  0.5799999833106995
Batch :  17 / 17 , BCE in this minibatch:  2.1206255 accuracy:  0.5839999914169312
epoch: 5  train_loss: 2.119290225646075  accuracy:  0.5839999914169312
Batch :  1 / 17 , BCE in this minibatch:  2.1586578 accuracy:  0.5669999718666077
Batch :  2 / 17 , BCE in this minibatch:  2.1674001 accuracy:  0.5770000219345093
Batch :  3 / 17 , BCE in this minibatch:  2.0860202 accuracy:  0.5640000104904175
Batch :  4 / 17 , BCE in this minibatch:  2.212872 accuracy:  0.5740000009536743
Batch :  5 / 17 , BCE in this minibatch:  2.006899 accuracy:  0.593999981880188
Batch :  6 / 17 , BCE in this minibatch:  1.8824565 accuracy:  0.6209999918937683
Batch :  7 / 17 , BCE in 

Batch :  6 / 17 , BCE in this minibatch:  1.577287 accuracy:  0.6539999842643738
Batch :  7 / 17 , BCE in this minibatch:  1.6975234 accuracy:  0.6259999871253967
Batch :  8 / 17 , BCE in this minibatch:  1.7921858 accuracy:  0.6060000061988831
Batch :  9 / 17 , BCE in this minibatch:  1.7138911 accuracy:  0.6200000047683716
Batch :  10 / 17 , BCE in this minibatch:  1.6710111 accuracy:  0.6290000081062317
Batch :  11 / 17 , BCE in this minibatch:  1.482236 accuracy:  0.656000018119812
Batch :  12 / 17 , BCE in this minibatch:  1.6499956 accuracy:  0.6380000114440918
Batch :  13 / 17 , BCE in this minibatch:  1.6368483 accuracy:  0.6240000128746033
Batch :  14 / 17 , BCE in this minibatch:  1.708387 accuracy:  0.6439999938011169
Batch :  15 / 17 , BCE in this minibatch:  1.6156349 accuracy:  0.6520000100135803
Batch :  16 / 17 , BCE in this minibatch:  1.6903008 accuracy:  0.628000020980835
Batch :  17 / 17 , BCE in this minibatch:  1.7354338 accuracy:  0.6460000276565552
epoch: 11  tr

Batch :  17 / 17 , BCE in this minibatch:  1.5199404 accuracy:  0.6700000166893005
epoch: 16  train_loss: 1.4706743254381067  accuracy:  0.6700000166893005
Batch :  1 / 17 , BCE in this minibatch:  1.4971493 accuracy:  0.6489999890327454
Batch :  2 / 17 , BCE in this minibatch:  1.5531774 accuracy:  0.6610000133514404
Batch :  3 / 17 , BCE in this minibatch:  1.4937407 accuracy:  0.6669999957084656
Batch :  4 / 17 , BCE in this minibatch:  1.546102 accuracy:  0.6510000228881836
Batch :  5 / 17 , BCE in this minibatch:  1.4267834 accuracy:  0.6629999876022339
Batch :  6 / 17 , BCE in this minibatch:  1.3328031 accuracy:  0.6800000071525574
Batch :  7 / 17 , BCE in this minibatch:  1.428873 accuracy:  0.6679999828338623
Batch :  8 / 17 , BCE in this minibatch:  1.4992812 accuracy:  0.6620000004768372
Batch :  9 / 17 , BCE in this minibatch:  1.4361217 accuracy:  0.6549999713897705
Batch :  10 / 17 , BCE in this minibatch:  1.4091765 accuracy:  0.6710000038146973
Batch :  11 / 17 , BCE in

Batch :  11 / 17 , BCE in this minibatch:  1.0653808 accuracy:  0.7179999947547913
Batch :  12 / 17 , BCE in this minibatch:  1.2252041 accuracy:  0.6959999799728394
Batch :  13 / 17 , BCE in this minibatch:  1.2426939 accuracy:  0.6830000281333923
Batch :  14 / 17 , BCE in this minibatch:  1.3233074 accuracy:  0.6980000138282776
Batch :  15 / 17 , BCE in this minibatch:  1.2118046 accuracy:  0.699999988079071
Batch :  16 / 17 , BCE in this minibatch:  1.305384 accuracy:  0.6899999976158142
Batch :  17 / 17 , BCE in this minibatch:  1.3292167 accuracy:  0.6890000104904175
epoch: 22  train_loss: 1.2785258152905632  accuracy:  0.6890000104904175
Batch :  1 / 17 , BCE in this minibatch:  1.3000782 accuracy:  0.6869999766349792
Batch :  2 / 17 , BCE in this minibatch:  1.3732877 accuracy:  0.6859999895095825
Batch :  3 / 17 , BCE in this minibatch:  1.3215532 accuracy:  0.6959999799728394
Batch :  4 / 17 , BCE in this minibatch:  1.3413336 accuracy:  0.6759999990463257
Batch :  5 / 17 , BC

...

Batch :  2 / 17 , BCE in this minibatch:  0.73404825 accuracy:  0.781000018119812
Batch :  3 / 17 , BCE in this minibatch:  0.7334697 accuracy:  0.7710000276565552
Batch :  4 / 17 , BCE in this minibatch:  0.6663103 accuracy:  0.7839999794960022
Batch :  5 / 17 , BCE in this minibatch:  0.67467904 accuracy:  0.7760000228881836
Batch :  6 / 17 , BCE in this minibatch:  0.6377209 accuracy:  0.7910000085830688
Batch :  7 / 17 , BCE in this minibatch:  0.6521599 accuracy:  0.7979999780654907
Batch :  8 / 17 , BCE in this minibatch:  0.6693331 accuracy:  0.7760000228881836
Batch :  9 / 17 , BCE in this minibatch:  0.6176926 accuracy:  0.7850000262260437
Batch :  10 / 17 , BCE in this minibatch:  0.68157035 accuracy:  0.777999997138977
Batch :  11 / 17 , BCE in this minibatch:  0.548343 accuracy:  0.7960000038146973
Batch :  12 / 17 , BCE in this minibatch:  0.5778807 accuracy:  0.7929999828338623
Batch :  13 / 17 , BCE in this minibatch:  0.6319803 accuracy:  0.7680000066757202
Batch :  14 

Batch :  13 / 17 , BCE in this minibatch:  0.6038803 accuracy:  0.7760000228881836
Batch :  14 / 17 , BCE in this minibatch:  0.6724475 accuracy:  0.7799999713897705
Batch :  15 / 17 , BCE in this minibatch:  0.5806587 accuracy:  0.7799999713897705
Batch :  16 / 17 , BCE in this minibatch:  0.6596103 accuracy:  0.7839999794960022
Batch :  17 / 17 , BCE in this minibatch:  0.66299444 accuracy:  0.7730000019073486
epoch: 78  train_loss: 0.6304339170455933  accuracy:  0.7730000019073486
Batch :  1 / 17 , BCE in this minibatch:  0.62988967 accuracy:  0.7950000166893005
Batch :  2 / 17 , BCE in this minibatch:  0.6959532 accuracy:  0.7850000262260437
Batch :  3 / 17 , BCE in this minibatch:  0.7014596 accuracy:  0.777999997138977
Batch :  4 / 17 , BCE in this minibatch:  0.628396 accuracy:  0.7950000166893005
Batch :  5 / 17 , BCE in this minibatch:  0.64340514 accuracy:  0.7829999923706055
Batch :  6 / 17 , BCE in this minibatch:  0.6085824 accuracy:  0.7979999780654907
Batch :  7 / 17 , B

Batch :  6 / 17 , BCE in this minibatch:  0.5864819 accuracy:  0.8059999942779541
Batch :  7 / 17 , BCE in this minibatch:  0.59647644 accuracy:  0.8019999861717224
Batch :  8 / 17 , BCE in this minibatch:  0.6052667 accuracy:  0.7839999794960022
Batch :  9 / 17 , BCE in this minibatch:  0.57149744 accuracy:  0.796999990940094
Batch :  10 / 17 , BCE in this minibatch:  0.62526125 accuracy:  0.7929999828338623
Batch :  11 / 17 , BCE in this minibatch:  0.5034761 accuracy:  0.8040000200271606
Batch :  12 / 17 , BCE in this minibatch:  0.5316472 accuracy:  0.7910000085830688
Batch :  13 / 17 , BCE in this minibatch:  0.5767074 accuracy:  0.781000018119812
Batch :  14 / 17 , BCE in this minibatch:  0.6448543 accuracy:  0.7770000100135803
Batch :  15 / 17 , BCE in this minibatch:  0.5543598 accuracy:  0.7860000133514404
Batch :  16 / 17 , BCE in this minibatch:  0.6275704 accuracy:  0.7940000295639038
Batch :  17 / 17 , BCE in this minibatch:  0.6352626 accuracy:  0.7829999923706055
epoch: 

Batch :  16 / 17 , BCE in this minibatch:  0.6035346 accuracy:  0.7979999780654907
Batch :  17 / 17 , BCE in this minibatch:  0.6151493 accuracy:  0.7900000214576721
epoch: 89  train_loss: 0.5799309278235716  accuracy:  0.7900000214576721
Batch :  1 / 17 , BCE in this minibatch:  0.57661295 accuracy:  0.800000011920929
Batch :  2 / 17 , BCE in this minibatch:  0.6401575 accuracy:  0.7929999828338623
Batch :  3 / 17 , BCE in this minibatch:  0.6526537 accuracy:  0.7879999876022339
Batch :  4 / 17 , BCE in this minibatch:  0.5624885 accuracy:  0.8040000200271606
Batch :  5 / 17 , BCE in this minibatch:  0.5947867 accuracy:  0.7910000085830688
Batch :  6 / 17 , BCE in this minibatch:  0.5626094 accuracy:  0.8029999732971191
Batch :  7 / 17 , BCE in this minibatch:  0.57069325 accuracy:  0.8050000071525574
Batch :  8 / 17 , BCE in this minibatch:  0.5699598 accuracy:  0.7839999794960022
Batch :  9 / 17 , BCE in this minibatch:  0.5505844 accuracy:  0.8050000071525574
Batch :  10 / 17 , BCE

Batch :  9 / 17 , BCE in this minibatch:  0.5349073 accuracy:  0.8090000152587891
Batch :  10 / 17 , BCE in this minibatch:  0.5802805 accuracy:  0.8080000281333923
Batch :  11 / 17 , BCE in this minibatch:  0.46648917 accuracy:  0.8069999814033508
Batch :  12 / 17 , BCE in this minibatch:  0.49712056 accuracy:  0.800000011920929
Batch :  13 / 17 , BCE in this minibatch:  0.537725 accuracy:  0.7889999747276306
Batch :  14 / 17 , BCE in this minibatch:  0.6023915 accuracy:  0.7820000052452087
Batch :  15 / 17 , BCE in this minibatch:  0.5145782 accuracy:  0.796999990940094
Batch :  16 / 17 , BCE in this minibatch:  0.577812 accuracy:  0.8069999814033508
Batch :  17 / 17 , BCE in this minibatch:  0.5922445 accuracy:  0.7910000085830688
epoch: 95  train_loss: 0.5574366713271421  accuracy:  0.7910000085830688
Batch :  1 / 17 , BCE in this minibatch:  0.55359334 accuracy:  0.8050000071525574
Batch :  2 / 17 , BCE in this minibatch:  0.6161078 accuracy:  0.796999990940094
Batch :  3 / 17 , B